# Nominal_NETs

In [1]:
import numpy as np
import yaml
import toml
from datetime import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline  
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [12, 20]
plt.rc('xtick',labelsize=10)
plt.rc('ytick',labelsize=10)
mpl.rc('xtick', direction='in', top=True)
mpl.rc('ytick', direction='in', right=True)
mpl.rc('xtick.minor', visible=True)
mpl.rc('ytick.minor', visible=True)
from scipy.interpolate import InterpolatedUnivariateSpline
import pandas as pd
from IPython.core.display import display, HTML

def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

# bolo-calc import
from bolo import Top

In [2]:
def NET_TeleTable(telescope, in_param, start, end, npts, percent):
    out_param = 'NET'
    param_vec = np.linspace(start, end, npts)
    files = ['SAT_pole_20210401', 'SPLAT_20210401', 'CHLAT_20210401']
#     psat = {}
#     psat['SAT_pole_20210401'] = {'LF_1': 1.40, 'LF_2': 6.13, 'MF1_1': 7.40, 'MF1_2': 12.25, 'MF2_1': 7.65, 'MF2_2': 13.30, 'UHF_1': 29.88, 'UHF_2': 39.03}
#     psat['SPLAT_20210401'] = {'ULF': 0.42, 'LF_1': 0.69, 'LF_2': 4.32, 'MF_1': 4.65, 'MF_2': 11.49, 'UHF_1': 29.31, 'UHF_2': 40.02}
#     psat['CHLAT_20210401'] = {'LF_1': 0.72, 'LF_2': 3.84, 'MF_1': 4.47, 'MF_2': 13.95, 'UHF_1': 41.52, 'UHF_2': 60.78}
    yaml_file, dd, nominal, ch_names, outputs, top, tabs, Net = \
    {},{},{},{},{},{},{},{}
    yaml_file =  f'{telescope}_20210401.yaml'
    dd = yaml.safe_load(open(yaml_file))
    del dd['version']
    ch_names = list(dd['instrument']['camera_config']['elements']['cam_1']['chan_config']['elements'].keys())
    nominal = {}
    nominal = dd['instrument']['channel_default'][in_param]
    outputs[out_param] = {}
    for chan in ch_names:
        outputs[out_param][chan] = [] 
#         if in_param != 'psat_factor':
#             psat[f'{telescope}_20210401'][chan] *= 1e-12
#             dd['instrument']['camera_config']['elements']['cam_1']['chan_config']['elements'][chan]['psat'] = psat[f'{telescope}_20210401'][chan]
#     if in_param != 'psat_factor':     
#         dd['instrument']['channel_default']['psat_factor'] = None
#         del dd['instrument']['channel_default']['psat']
    dd['sim_config']['config_dir'] = '../../bolo-calc/config'
    for param_value in param_vec:
        dd['instrument']['channel_default'][in_param] = param_value
        top = Top(**dd)
        top.run()
        tabs = top.instrument.tables
        for chan in ch_names:
            outputs[out_param][chan] = np.append(outputs[out_param][chan],\
                                                    tabs['cam_1_%s_sims' % chan][out_param])
    ypv,left_yv,right_yv,slope,NET,ynv,\
    left,right,pxvp,nxvp,label,left_slope,right_slope = {},{},{},{},{},{},{},{},{},{},{},{},{}
    vals, data = [],[]   
    for chan in ch_names:
        Net[chan] = outputs[out_param][chan]
        ynv[chan] = np.interp(nominal,param_vec,Net[chan])#y-value for nominal
        left = np.where(param_vec<nominal)[0] #left of nominal, holds index locations
        left_xv = param_vec[left]
        left_yv[chan] = Net[chan][left]  #The corresponding y-values
        right = np.where(param_vec>nominal)[0] #right of and including nominal, holds index locations
        right_xv = param_vec[right]
        right_yv[chan] = Net[chan][right] #The corresponding y-values
        left_slope[chan] = (ynv[chan]-left_yv[chan][-1])/(nominal-left_xv[-1]) #slope from the left
        right_slope[chan] = (right_yv[chan][0]-ynv[chan])/(right_xv[0]-nominal) #slope from the right
        ypv[chan] = {}
        pxvp[chan] = {}
        nxvp[chan] = {}
        slope[chan] = {}
        step = 1e-6 #step size in finding slope
        slope[chan] = (left_slope[chan]+right_slope[chan])/2
#         slope[chan] = ( np.interp((nominal+step),right_xv[chan],right_yv[chan]) - ynv[chan])/((nominal[chan]+step) - nominal[chan])
        vals.append([])
        for per in percent:
            ypv[chan][per] = ynv[chan]*((100+per)/100) #y values for each positive
                                                      #percent for every aspect of output
            pxvp[chan][per] = np.interp(ypv[chan][per], right_yv[chan],right_xv)#right_xv[chan][np.where(right_yv[chan] >= ypv[chan][per])][0] #the corresponding pos values in x
            nxvp[chan][per] = np.interp(ypv[chan][per], np.flip(left_yv[chan]),np.flip(left_xv))#left_xv[chan][np.where( left_yv[chan] >= ypv[chan][per])][-1] #the corresponding neg values in x
#       The if loops make it so that if the x values are out of range of the input range, they say 'NaN'
            if ((nxvp[chan][per] == start) or (nxvp[chan][per] < start) or (nxvp[chan][per] > end) or (nxvp[chan][per]==end)) :
                nxvp[chan][per] = 'NaN' #Out of range
            if ((pxvp[chan][per] == start) or (pxvp[chan][per] < start) or (pxvp[chan][per] > end) or (pxvp[chan][per]==end)) :
                pxvp[chan][per] = 'NaN' #Out of range 
    for k in range(len(ch_names)):
        NET[ch_names[k]] = {}
        for p in range(len(percent)):
            label[p] = f'neg_{percent[len(percent)-p-1]}' #Label for the table 
                                                           #This part does the negative
            NET[ch_names[k]][label[p]] = nxvp[ch_names[k]][percent[-1-p]] #NET dictionary for output
            vals[k].append(nxvp[ch_names[k]][percent[-1-p]]) #Values for the table
    #This part does the nominal
    label[len(percent)] = 'nominal'
    label[len(percent)+1] = 'slope'
    for k in range(len(ch_names)):
        NET[ch_names[k]][label[len(percent)]] = nominal
        NET[ch_names[k]][label[len(percent)+1]] = slope[ch_names[k]]
        vals[k].append(nominal)
        vals[k].append(slope[ch_names[k]])
        for p in range(len(percent)):
            label[len(percent)+2+p] = f'pos_{percent[p]}'
            NET[ch_names[k]][label[len(percent)+2+p]] = pxvp[ch_names[k]][percent[p]]
            vals[k].append(pxvp[ch_names[k]][percent[p]])
        title = [] #matrix of the label to put in the table
        row = []
        for l in range(len(label)):
            title.append(label[l])
            row.append(vals[k][l])
        data.append(row)
    df = pd.DataFrame(data,index=ch_names, columns=title)
    return df, NET#ynv,left_xv,left_yv,right_xv,right_yv#df, NET

In [3]:
npts = 10
param_dict = {}
in_put = ['Tc','carrier_index', 'psat_factor']
start = [0.14,0,1.5]
end = [0.2,4,4]

# {'startval':0.14,'endval':0.2,'npts':npts,'nominal':}

for i in range(len(in_put)):
    param_dict[in_put[i]]= {}
    for s in range(len(start)):
        param_dict[in_put[i]]['startval']= start[i]
        param_dict[in_put[i]]['endval']= end[i]
        param_dict[in_put[i]]['npts']= npts

param_dict

{'Tc': {'startval': 0.14, 'endval': 0.2, 'npts': 10},
 'carrier_index': {'startval': 0, 'endval': 4, 'npts': 10},
 'psat_factor': {'startval': 1.5, 'endval': 4, 'npts': 10}}

In [4]:
%%time
pnts = 50

teles = ['SAT_pole', 'SPLAT','CHLAT']
in_put = ['Tc','carrier_index', 'psat_factor'] #Look in the yaml files
percents = [0.3,0.5,1]
dfTc = []
dfn = []
dfpsat = []

for i in range(len(teles)):
    dfTc.append([])
    dfn.append([])
    dfpsat.append([])
    dfTc[i] = NET_TeleTable(teles[i], in_put[0], 0.14, 0.2, pnts, percents)[0] #Tc: 0.14-0.2
    dfn[i] = NET_TeleTable(teles[i], in_put[1], 0, 4, pnts, percents)[0] #n: 0-4
    dfpsat[i] = NET_TeleTable(teles[i], in_put[2], 1.5, 4, pnts, percents)[0] #psat_factor: 1.5-4

CPU times: user 58.8 s, sys: 538 ms, total: 59.3 s
Wall time: 1min


In [5]:
display_side_by_side([dfTc[0], dfTc[1], dfTc[2]],\
                     [f' {in_put[0]}  -  {teles[0]}',f' {in_put[0]}  -  {teles[1]}', f' {in_put[0]}  -  {teles[2]}'])
display_side_by_side([dfn[0], dfn[1], dfn[2]],\
                     [f' {in_put[1]}  -  {teles[0]}',f' {in_put[1]}  -  {teles[1]}', f' {in_put[1]}  -  {teles[2]}'])
display_side_by_side([dfpsat[0], dfpsat[1], dfpsat[2]],\
                     [f' {in_put[2]}  -  {teles[0]}',f' {in_put[2]}  -  {teles[1]}', f' {in_put[2]}  -  {teles[2]}'])

,neg_1,neg_0.5,neg_0.3,nominal,slope,pos_0.3,pos_0.5,pos_1
LF_1,0.145953,0.151548,0.154429,0.160000,-88.143687,NaN,NaN,NaN
LF_2,NaN,0.146088,0.150374,0.160000,-46.727368,NaN,NaN,NaN
MF1_1,NaN,0.145718,0.150076,0.160000,-62.278747,NaN,NaN,NaN
MF1_2,NaN,0.143891,0.148631,0.160000,-50.128304,NaN,NaN,NaN
MF2_1,NaN,0.145960,0.150270,0.160000,-54.860829,NaN,NaN,NaN
MF2_2,NaN,0.143507,0.148329,0.160000,-51.407933,NaN,NaN,NaN
UHF_1,NaN,0.140250,0.145660,0.160000,-75.370156,NaN,NaN,NaN
UHF_2,NaN,NaN,0.143729,0.160000,-154.318004,NaN,NaN,NaN
,neg_1,neg_0.5,neg_0.3,nominal,slope,pos_0.3,pos_0.5,pos_1
ULF,0.151213,0.155036,0.156844,0.160000,-274.858547,NaN,NaN,NaN


,neg_1,neg_0.5,neg_0.3,nominal,slope,pos_0.3,pos_0.5,pos_1
LF_1,NaN,NaN,NaN,2.000000,2.434676,2.238138,2.390914,2.755340
LF_2,NaN,NaN,NaN,2.000000,1.290744,2.459650,2.744975,3.405267
MF1_1,NaN,NaN,NaN,2.000000,1.720321,2.477420,2.773301,3.456090
MF1_2,NaN,NaN,NaN,2.000000,1.384700,2.568298,2.916278,3.712222
MF2_1,NaN,NaN,NaN,2.000000,1.515415,2.465814,2.754800,3.423002
MF2_2,NaN,NaN,NaN,2.000000,1.420050,2.588540,2.948018,3.768651
UHF_1,NaN,NaN,NaN,2.000000,2.081980,2.777691,3.242122,NaN
UHF_2,NaN,NaN,NaN,2.000000,4.262808,2.929265,3.475614,NaN
,neg_1,neg_0.5,neg_0.3,nominal,slope,pos_0.3,pos_0.5,pos_1
ULF,NaN,NaN,NaN,2.000000,7.591446,2.126409,2.208904,2.409753


,neg_1,neg_0.5,neg_0.3,nominal,slope,pos_0.3,pos_0.5,pos_1
LF_1,NaN,NaN,NaN,2.500000,8.566825,2.569479,2.615926,2.732439
LF_2,NaN,NaN,NaN,2.500000,4.541203,2.637281,2.729027,2.959205
MF1_1,NaN,NaN,NaN,2.500000,6.052549,2.642874,2.738361,2.977907
MF1_2,NaN,NaN,NaN,2.500000,4.871657,2.671659,2.786391,3.074210
MF2_1,NaN,NaN,NaN,2.500000,5.331649,2.639221,2.732265,2.965692
MF2_2,NaN,NaN,NaN,2.500000,4.996006,2.678132,2.797188,3.095857
UHF_1,NaN,NaN,NaN,2.500000,7.324641,2.739817,2.900095,3.302185
UHF_2,NaN,NaN,NaN,2.500000,14.996867,2.790792,2.985136,3.472695
,neg_1,neg_0.5,neg_0.3,nominal,slope,pos_0.3,pos_0.5,pos_1
ULF,NaN,NaN,NaN,3.000000,22.247177,3.043679,3.072870,3.146117


In [6]:
%%time
NET = {}
pnts = 100

teles = ['SAT_pole', 'SPLAT','CHLAT']

in_put = ['Tc'  ,   'carrier_index',    'psat_factor']
bgns   = [0.14  ,        2         ,     1.5         ]
ends   = [0.2   ,        4         ,     4           ]


for i in range(len(teles)):
    NET[teles[i]] = {}
    for j in range(len(in_put)):
        NET[teles[i]][in_put[j]] = {}
        NET[teles[i]][in_put[j]] = NET_TeleTable2(teles[i], in_put[j], bgns[j], ends[j], pnts, [1,5,10,15])[1]

NameError: name 'NET_TeleTable2' is not defined

In [7]:
NET#['SAT_pole']['psat_factor']['MF1_1']['nominal']

{'SAT_pole': {'Tc': {}}}